In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv('/Users/vighneshms/Downloads/Email_classifier/models/masked_emails_with_types.csv')  # Assuming CSV format

# Explore the data
print(df.head())
print(df['type'].value_counts())  # Check class distribution

                                        masked_email      type
0  Subject: unforeseen crash of the data analysis...  Incident
1  Subject: Customer Support Inquiry\n\nSeeking i...   Request
2  Subject: Data Analytics for Investment\n\nI am...   Request
3  Subject: Hospital service problem\n\nMedia dat...  Incident
4  Subject: Security\n\nDear Customer Support, I ...   Request
type
Incident    9586
Request     6860
Problem     5037
Change      2517
Name: count, dtype: int64


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenize
    words = text.split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

df['processed_text'] = df['masked_email'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vighneshms/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vighneshms/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2) 
) # Include bigrams
X = tfidf.fit_transform(df['processed_text'])
y = df['type']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [7]:
rf = RandomForestClassifier(
    class_weight='balanced',  # Add class weights
    n_estimators=200,
    max_depth=None,
    min_samples_split=10,
    random_state=42
)
rf.fit(X_res, y_res)
rf_pred = rf.predict(X_test)
print("Random Forest:\n", classification_report(y_test, rf_pred))

Random Forest:
               precision    recall  f1-score   support

      Change       0.92      0.82      0.87       479
    Incident       0.72      0.92      0.81      1920
     Problem       0.71      0.35      0.47      1009
     Request       0.92      0.93      0.93      1392

    accuracy                           0.79      4800
   macro avg       0.82      0.76      0.77      4800
weighted avg       0.79      0.79      0.78      4800



In [8]:
import pickle

# Save the best model and vectorizer
with open('email_classifier.pkl', 'wb') as f:
    pickle.dump(rf, f)  # Assuming RF was best

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [7]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define base models
estimators = [
    ('svm', SVC(kernel='linear', probability=True)),
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier())
]

# Create stack with logistic regression as final estimator
stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(class_weight='balanced'),
    cv=5  # Use cross-validated predictions
)

# Train and evaluate
stack.fit(X_res, y_res)
y_pred = stack.predict(X_test)
print("Stacking:\n", classification_report(y_test, y_pred))

Stacking:
               precision    recall  f1-score   support

      Change       0.91      0.83      0.87       479
    Incident       0.71      0.86      0.78      1920
     Problem       0.61      0.40      0.48      1009
     Request       0.92      0.92      0.92      1392

    accuracy                           0.78      4800
   macro avg       0.79      0.75      0.76      4800
weighted avg       0.77      0.78      0.77      4800



In [7]:
import pandas as pd
df = pd.read_csv('/Users/vighneshms/Downloads/Email_classifier/models/emails_subject_body_type3.csv')  

In [8]:
from sklearn.model_selection import train_test_split

# Subject-only classification
X_subject = df['subject']
y_subject = df['type']

# Body-only classification
X_body = df['body']
y_body = df['type']

# Split data (70% train, 30% test)
X_subj_train, X_subj_test, y_subj_train, y_subj_test = train_test_split(
    X_subject, y_subject, test_size=0.3, random_state=42
)

X_body_train, X_body_test, y_body_train, y_body_test = train_test_split(
    X_body, y_body, test_size=0.3, random_state=42
)

In [10]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Shared preprocessing steps
preprocessor = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),  # Includes unigrams and bigrams
    ))
])

# Subject classifier
subject_clf = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', SVC(
        kernel='linear',
        class_weight='balanced',
        probability=True
    ))
])

# Body classifier
body_clf = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', SVC(
        kernel='linear',
        class_weight='balanced',
        probability=True
    ))
])

In [ ]:
# Train subject classifier
# subject_clf.fit(X_subj_train, y_subj_train)
# y_subj_pred = subject_clf.predict(X_subj_test)

# Train body classifier
# body_clf.fit(X_body_train, y_body_train)
# y_body_pred = body_clf.predict(X_body_test)

# Compare performance
# print("=== Subject-Only Performance ===")
# print(classification_report(y_subj_test, y_subj_pred))

# print("\n=== Body-Only Performance ===")
# print(classification_report(y_body_test, y_body_pred))


=== Body-Only Performance ===
              precision    recall  f1-score   support

      Change       0.79      0.86      0.82       762
    Incident       0.74      0.61      0.67      2869
     Problem       0.44      0.59      0.50      1484
     Request       0.93      0.89      0.91      2085

    accuracy                           0.71      7200
   macro avg       0.72      0.74      0.72      7200
weighted avg       0.74      0.71      0.72      7200



In [11]:
# Train subject classifier
subject_clf.fit(X_subj_train, y_subj_train)
y_subj_pred = subject_clf.predict(X_subj_test)

print("=== Subject-Only Performance ===")
print(classification_report(y_subj_test, y_subj_pred))

=== Subject-Only Performance ===
              precision    recall  f1-score   support

      Change       0.43      0.78      0.56       762
    Incident       0.71      0.53      0.60      2869
     Problem       0.41      0.48      0.44      1484
     Request       0.83      0.76      0.80      2085

    accuracy                           0.61      7200
   macro avg       0.59      0.64      0.60      7200
weighted avg       0.65      0.61      0.62      7200

